In [42]:
## Importing Libraries

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

print(tf.__version__)

tf.set_random_seed(777)

1.9.0


In [43]:
## training에 사용할 image size
img_width = 128
img_height = 128

In [44]:
## tfrecord file 이름과 디렉토리 설정
tfrecord_train = 'train.tfrecord'
tfrecord_test = 'test.tfrecord'
tfrecord_dir = './tfrecords'

In [45]:
## tfrecord file 경로 설정
cur_dir = os.getcwd()
train_tfr_path = os.path.join(cur_dir, tfrecord_dir, tfrecord_train)
test_tfr_path = os.path.join(cur_dir, tfrecord_dir, tfrecord_test)

In [46]:
## hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 16

## class 갯수
n_class = 101
## training data 수
n_train = 6941
## test data 수
n_test = 1736
## learning rate decay ratio
lr_decay_ratio = 0.1
## 몇 epoch 마다 learning rate을 decay할 것인지
lr_decay_epoch_num = 5
## image file 위치
image_dir = './data/caltech101'

## image file 경로에 directory가 몇 개 있는지(class 수) 확인
print ('{} categories in {}'.format(len(os.listdir(image_dir)), image_dir))
categories = sorted(os.listdir(image_dir))
num_categ = len(categories)
print (categories)

101 categories in ./data/caltech101
['Faces', 'Faces_easy', 'Leopards', 'Motorbikes', 'accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion', 'sea_horse

In [47]:
def _parse_function(tfrecord_serialized):
    features={'image': tf.FixedLenFeature([], tf.string),
             'label': tf.FixedLenFeature([], tf.int64)}
    parsed_features = tf.parse_single_example(tfrecord_serialized, features)
    
    image = tf.decode_raw(parsed_features['image'], tf.uint8)
    label = tf.cast(parsed_features['label'], tf.int32)
    #label_onehot = tf.one_hot(label, depth=n_class)
        
    #image = tf.reshape(image, [-1, img_height, img_width, 3])
    
    return image, label

In [59]:
train_dataset = tf.data.TFRecordDataset(train_tfr_path)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=4)
train_dataset = train_dataset.shuffle(buffer_size=n_train*2).prefetch(buffer_size=batch_size).batch(batch_size).repeat()
print(train_dataset.output_types, train_dataset.output_shapes)

(tf.uint8, tf.int32) (TensorShape([Dimension(None), Dimension(None)]), TensorShape([Dimension(None)]))


In [60]:
test_dataset = tf.data.TFRecordDataset(test_tfr_path)
test_dataset = test_dataset.map(_parse_function, num_parallel_calls=4)
test_dataset = test_dataset.shuffle(buffer_size=n_train*2).prefetch(buffer_size=batch_size).batch(batch_size).repeat()
print(test_dataset.output_types, test_dataset.output_shapes)

(tf.uint8, tf.int32) (TensorShape([Dimension(None), Dimension(None)]), TensorShape([Dimension(None)]))


In [50]:
iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
image, label = iterator.get_next()

In [51]:
train_init = iterator.make_initializer(train_dataset)
test_init = iterator.make_initializer(test_dataset)

In [52]:
image_post = tf.reshape(image, [-1, img_height, img_width, 3])
image_post = tf.cast(image_post, tf.float32) / 255.
label_onehot = tf.one_hot(label, depth=n_class)
is_train = tf.placeholder(tf.bool)

In [53]:
conv1 = tf.layers.conv2d(inputs=image_post, filters=32, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                        padding="SAME", strides=2)
bn1 = tf.layers.batch_normalization(pool1, training=is_train)
#bn1 = tf.contrib.layers.batch_norm(pool1, decay=0.9, is_training=is_train)
#drop1 = tf.layers.dropout(inputs=pool1, rate=0.3, training=is_train)

conv2 = tf.layers.conv2d(inputs=bn1, filters=64, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                        padding="SAME", strides=2)
bn2 = tf.layers.batch_normalization(pool2, training=is_train)
#bn2 = tf.contrib.layers.batch_norm(pool2, decay=0.9, is_training=is_train)
#drop2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=is_train)

conv3 = tf.layers.conv2d(inputs=bn2, filters=128, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                        padding="SAME", strides=2)
bn3 = tf.layers.batch_normalization(pool3, training=is_train)
#bn3 = tf.contrib.layers.batch_norm(pool3, decay=0.9, is_training=is_train)
#drop3 = tf.layers.dropout(inputs=pool3, rate=0.3, training=is_train)

In [54]:
flat3 = tf.contrib.layers.flatten(bn3)
dense4 = tf.layers.dense(inputs=flat3, units=625, activation=tf.nn.relu)
bn4 = tf.layers.batch_normalization(dense4, training=is_train)
#bn4 = tf.contrib.layers.batch_norm(dense4, decay=0.9, is_training=is_train)
#drop4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=is_train)

logits = tf.layers.dense(inputs=bn4, units=n_class)

In [55]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=label_onehot))
global_step = tf.Variable(0, trainable=False)
lr_decay = tf.train.exponential_decay(learning_rate=learning_rate,
                                          global_step= global_step,
                                          decay_steps=int(n_train/batch_size*lr_decay_epoch_num),
                                          decay_rate= lr_decay_ratio,
                                          staircase=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(
        cost)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [56]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label_onehot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [57]:
# initialize
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())

In [58]:
# train my model
print('Learning started. It takes sometime.')
max_test_acc = 0.
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    
    total_batch = int(n_train / batch_size)
    total_batch_test = int(n_test / batch_size)
    
    sess.run(train_init)

    for i in range(total_batch):
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        #sess.run([image, label])        
        feed_dict = {is_train:True}
        acc, c, _ = sess.run([accuracy, cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
        
    sess.run(test_init)
        
    for i in range(total_batch_test):
        #batch_xs, batch_ys = mnist.test.next_batch(batch_size)        
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        #sess.run([image, label])
        feed_dict = {is_train:False}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_test_acc += acc / total_batch_test

    print('Epoch:', '{}'.format(epoch + 1), 'cost =', '{:.8f}'.format(avg_cost), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))

print('Learning Finished!')

Learning started. It takes sometime.


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype bool
	 [[Node: Placeholder_1 = Placeholder[dtype=DT_BOOL, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_1', defined at:
  File "/home/sungwookcho/anaconda2/envs/tf_keras_py3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sungwookcho/anaconda2/envs/tf_keras_py3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sungwookcho/anaconda2/envs/tf_keras_py3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/sungwookcho/anaconda2/envs/tf_keras_py3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sungwookcho/anaconda2/envs/tf_keras_py3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.s

In [ ]:
cur_dir = os.getcwd()
image_path = os.path.join(cur_dir, image_dir)

class_names = sorted(os.listdir(image_path))
print(class_names)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    #plt.xticks([])
    plt.xticks(range(n_class), class_names, rotation=90)
    plt.yticks([])
    thisplot = plt.bar(range(n_class), predictions_array, color="#777777")
    plt.ylim([0, 1]) 
    predicted_label = np.argmax(predictions_array)
 
    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
sess.run(test_init)

In [ ]:
prob = tf.nn.softmax(logits)
imgs, lbs, x, y = sess.run([image, label, image_post, label_onehot])
imgs = np.reshape(imgs, [-1, img_height, img_width, 3])
predictions = sess.run(prob, feed_dict={image_post:x, label_onehot:y, is_train:False})
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(3*2*num_cols, 3*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, lbs, imgs)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions, lbs)